In [ ]:
import json
import pandas as pd

In [ ]:
with open('../../data/meta_Electronics.jsonl', 'r') as f:
    first_line = json.loads(f.readline())

In [ ]:
# Display the first record to see the structure
print("First record structure:")
print(json.dumps(first_line, indent=2))

In [ ]:
# Show the keys available in the data
print("\nAvailable fields:")
print(list(first_line.keys()))

# Filter items that have observed for the first time in year 2022 or later

In [ ]:
def filter_data(data: dict) -> dict:
    filter = False
    if int(data['details']['Date First Available'][-4:]) < 2022:
        filter = True
    return filter

In [ ]:
with open("../../data/meta_Electronics.jsonl", 'r') as fp:
    with open("../../data/meta_Electronics_2022_2023.jsonl", 'w', encoding='utf-8') as fp_out:
        with open("../../data/meta_Electronics_2022_2023_no_date.jsonl", 'w', encoding='utf-8') as fp_out_no_date:
            i = 0
            for line in fp:
                data = json.loads(line.strip())
                try:
                    filter = filter_data(data)
                    if not filter:
                        json.dump(data, fp_out)
                        fp_out.write('\n')
                        fp_out.flush()
                except:
                    json.dump(data, fp_out_no_date)
                    fp_out_no_date.write('\n')
                    fp_out_no_date.flush()
                i += 1
                if i % 10000 == 0:
                    print(f"Processed {i} lines")

# Split the items into two categories: "has main category", "does not have main category"

In [ ]:
def filter_category(data: dict) -> dict:
    filter = False
    if data['main_category'] == None:
        filter = True

    return filter

In [ ]:
with open("../../data/meta_Electronics_2022_2023.jsonl", 'r') as fp:
    with open("../../data/meta_Electronics_2022_2023_with_category.jsonl", 'w', encoding='utf-8') as fp_out:
        with open("../../data/meta_Electronics_2022_2023_no_category.jsonl", 'w', encoding='utf-8') as fp_out_no_category:
            for line in fp:
                data = json.loads(line.strip())
                if not filter_category(data):
                    json.dump(data, fp_out)
                    fp_out.write('\n')
                    fp_out.flush()
                else:
                    json.dump(data, fp_out_no_category)
                    fp_out_no_category.write('\n')
                    fp_out_no_category.flush()

# Explore distribution by categories

In [ ]:
df = pd.read_json("../../data/meta_Electronics_2022_2023_with_category.jsonl",lines=True)

In [ ]:
# Display basic information about the DataFrame
print(f"Total records: {len(df)}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())

In [ ]:
# Show first few rows
df.head()

In [ ]:
# Explore category distribution
category_counts = df['main_category'].value_counts()
print("Distribution by main category:")
print(category_counts)

In [ ]:
df['main_category'].value_counts().plot(kind='bar', title='Distribution by Main Category')

# Filter out items that have at 100 ratings

In [ ]:
df_ratings = df[df['rating_number'] > 100]

In [ ]:
len(df)

In [ ]:
len(df_ratings)

In [ ]:
df_ratings['main_category'].value_counts().plot(kind='bar', title='Distribution by Main Category (Ratings > 100)')  

# Explore distribution of ratings

In [ ]:
df_ratings['average_rating'].plot(kind='hist', bins=50, title='Average Rating Distribution (Ratings > 100)')

# Sample 1000 items

In [ ]:
df_sample_1000 = df_ratings.sample(n=1000, random_state=20)

In [ ]:
df_sample_1000['average_rating'].plot(kind='hist', bins=50, range=(0, 5), title='Average Rating Distribution (Sample of 1000)')   

In [ ]:
df_sample_1000['main_category'].value_counts().plot(kind='bar', title='Distribution by Main Category (Sample of 1000)')

In [ ]:
df_ratings.to_json("../../data/meta_Electronics_2022_2023_with_category_ratings_over_100.jsonl", lines=True, orient='records')

In [ ]:
df_sample_1000.to_json("../../data/meta_Electronics_2022_2023_with_category_ratings_over_100_sample_1000.jsonl", lines=True, orient='records')

# Extract ratings that match sampled data

In [ ]:
df_ratings.to_json(
    "../../data/meta_Electronics_2022_2023_with_category_ratings_over_100.jsonl",
    lines=True,
    orient="records",
)
df_sample_1000.to_json(
    "../../data/meta_Electronics_2022_2023_with_category_ratings_over_100_sample_1000.jsonl",
    lines=True,
    orient="records",
)

In [ ]:
df_ratings_100 = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_over_100.jsonl", lines=True)
df_sample_1000 = pd.read_json("../../data/meta_Electronics_2022_2023_with_category_ratings_over_100_sample_1000.jsonl", lines=True)

In [ ]:
with open("../../data/Electronics.jsonl", 'r') as fp:
    with open("../../data/Electronics_2022_2023_with_category_ratings_100.jsonl", 'a') as fp_out:
        id_list = set(df_ratings_100['parent_asin'].values)
        i = 0
        for line in fp:
            data = json.loads(line.strip())
            if data['parent_asin'] in id_list:
                json.dump(data, fp_out)
                fp_out.write('\n')
                fp_out.flush()
            i += 1
            if i % 100000 == 0:
                print(f"Processed {i} lines")

In [ ]:
with open("../../data/Electronics_2022_2023_with_category_ratings_100.jsonl", 'r') as fp:
    with open("../../data/Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl", 'a') as fp_out:
        id_list = set(df_sample_1000['parent_asin'].values)
        i = 0
        for line in fp:
            data = json.loads(line.strip())
            if data['parent_asin'] in id_list:
                json.dump(data, fp_out)
                fp_out.write('\n')
                fp_out.flush()
            i += 1
            if i % 100000 == 0:
                print(f"Processed {i} lines")